# Machine Learning - Serverless Hosting

Machine Learning models can be hosted on server less functions. To host this model, we must consider the size of the packages and their security settings. 

In [ ]:
# download the file

!wget https://github.com/alexeygrigorev/large-datasets/releases/download/wasps-bees/bees-wasps.h5

In [6]:
import tensorflow as tf
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model

In [14]:

def convert_keras_model(path, output_path):
    """
    Convert a keras model to TensorFlow Lite format and save it to a file.
    """
    
    model = load_model(path)
   
    # Convert the model to TensorFlow Lite format   
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    lite_model = converter.convert()

    # Save the converted model to a file
    with open(output_path, "wb") as f:
        f.write(lite_model)
        # display the byte size of the model
        print("Size of the model(MB): ", round(len(lite_model) / 1024 / 1024, 0))

## Question 1 -  convert this model from Keras to TF-Lite format. What is the size?

What's the size of the converted model?

- 21 Mb
- 43 Mb
- 80 Mb
- 164 Mb


In [15]:
# convert the model
convert_keras_model('./models/bees-wasps.h5', './models/bees-wasps.tflite')


INFO:tensorflow:Assets written to: /tmp/tmphg9lkk78/assets


INFO:tensorflow:Assets written to: /tmp/tmphg9lkk78/assets
2023-11-21 16:19:19.750276: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-11-21 16:19:19.750374: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-11-21 16:19:19.750809: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmphg9lkk78
2023-11-21 16:19:19.751855: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-11-21 16:19:19.751936: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmphg9lkk78
2023-11-21 16:19:19.756514: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-11-21 16:19:19.843845: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmphg9lkk78
2023-11-21 16:19:19.855642: I tensorflow/cc/saved_model/loader.cc:314] SavedModel

Size of the model(MB):  43.0


## Question 2 - What's the output index for this model?

To be able to use this model, we need to know the index of the input and the index of the output.

What's the output index for this model?

- 3
- 7
- 13
- 24


In [3]:
# Image processing code

# pip install pillow 

from io import BytesIO
from urllib import request
from PIL import Image  

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess_image(img):
    """
    convert the image to a numpy array and normalize it
    """
    # convert to numpy array
    img = np.array(img)
    
    # convert to float32 to avoid overflow when multiplying by 255
    img = img.astype('float32')
    
    # normalize to the range 0-1
    img /= 255

    return img  


## Question 3 -  what's the value in the first pixel, the R channel?

Now we need to turn the image into numpy array and pre-process it.

Tip: Check the previous homework. What was the pre-processing we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

- 0.3450980
- 0.5450980
- 0.7450980
- 0.9450980


In [4]:
# download the image
image_url = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'

img_stream = download_image(image_url)
img = prepare_image(img_stream, target_size=(150, 150))

# convert the image to numpy array
img_normalized = preprocess_image(img)

# print the first pixel on the R channel (normalized)
print(img_normalized[0, 0, 0])
    

0.94509804


## Question 4 -  What's the output of the model?

Now let's apply this model to this image. What's the output of the model?

- 0.258
- 0.458
- 0.658
- 0.858

In [17]:
def load_lite_model(path):
    """
    Load the TensorFlow Lite model and allocate tensors.
    """
    # Load the TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path=path)
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()    

    return interpreter, input_details, output_details

In [21]:
# load the lite model and the input/output details
interpreter, input_details, output_details = load_lite_model('./models/bees-wasps.tflite')

# print the input shape and type
print(input_details[0]['shape'])
print(input_details[0]['dtype'])

# print the output shape and type
print(output_details[0]['shape'])
print(output_details[0]['dtype'])

# set the input tensor with the normalized image
interpreter.set_tensor(input_details[0]['index'], [img_normalized])

# run the inference
interpreter.invoke()

# get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# print the output
print('Tensor Output',round(output_data[0][0],3))

[  1 150 150   3]
<class 'numpy.float32'>
[1 1]
<class 'numpy.float32'>
Tensor Output 0.659
